<a href="https://colab.research.google.com/github/marcusflygar1-hash/AI_Course_Submissions_real/blob/main/Excersise_2_MarcusFlygar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sn
import matplotlib.pyplot as plt

In [5]:
url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_2_regression_model/Exercise2BusData.csv'
df = pd.read_csv(url)
df.head(10)

,Arrival_time,Stop_id,Bus_id,Line_id,Arrival_delay,Dwell_time,Scheduled_travel_time,Upstream_stop_delay,Recurrent_delay
0,01/05/2022 07:03,10033,41355,1,-23,0,39,-4,28.641509
1,01/05/2022 07:21,10033,41356,1,168,0,39,181,27.880597
2,01/05/2022 07:37,10033,44400,1,231,0,39,244,27.852941
3,01/05/2022 07:50,10033,44399,1,150,17,39,143,27.826087
4,01/05/2022 19:29,10033,44060,1,965,35,51,947,42.029412
5,01/05/2022 19:58,10033,41351,1,33,19,46,30,38.970588
6,01/05/2022 20:14,10033,44399,1,75,26,46,63,38.840580
7,01/05/2022 20:28,10033,41356,1,58,15,46,52,34.059701
8,01/05/2022 20:57,10033,44060,1,904,35,46,888,34.102941
9,01/05/2022 21:00,10033,41357,1,141,12,46,140,34.000000


In [6]:
df = df.drop(['Arrival_time', 'Stop_id','Bus_id','Line_id'], axis=1)
df.head(10)

,Arrival_delay,Dwell_time,Scheduled_travel_time,Upstream_stop_delay,Recurrent_delay
0,-23,0,39,-4,28.641509
1,168,0,39,181,27.880597
2,231,0,39,244,27.852941
3,150,17,39,143,27.826087
4,965,35,51,947,42.029412
5,33,19,46,30,38.970588
6,75,26,46,63,38.840580
7,58,15,46,52,34.059701
8,904,35,46,888,34.102941
9,141,12,46,140,34.000000


In [8]:
df.shape


(2179, 5)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Arrival_delay          2179 non-null   int64  
 1   Dwell_time             2179 non-null   int64  
 2   Scheduled_travel_time  2179 non-null   int64  
 3   Upstream_stop_delay    2179 non-null   int64  
 4   Recurrent_delay        2179 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 85.2 KB


In [13]:
corr_matrix = df.corr() #Creates a correlation matrix
corr_matrix['Arrival_delay'].sort_values(ascending=False) #The correlation matrix focuses on the correlation between the remaining columns to Arrival Delay

,Arrival_delay
Arrival_delay,1.000000
Upstream_stop_delay,0.993935
Dwell_time,0.218745
Recurrent_delay,0.162692
Scheduled_travel_time,0.100695


As we can see the upstream stop delay is heavily correlated to the arrival delay, however this is not true for the remaining independent variables.

In [16]:
x = df.drop('Arrival_delay', axis=1)
y = df['Arrival_delay']
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

The reason as to why we set the test_size = 0,2 is that this will devide the model into 80% training data and 20% of the data to test this on.

In [18]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

Now we have trained and tested the data set, now we will evaluate the model and see if it performs well.

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
lin_mse = mean_squared_error(y_test, y_pred)
lin_mae = mean_absolute_error(y_test, y_pred)
lin_rmse = root_mean_squared_error(y_test, y_pred)
lin_r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {lin_mse}")
print(f"Mean Absolute Error: {lin_mae}")
print(f"Root Mean Squared Error: {lin_rmse}")
print(f"R2 Score: {lin_r2}")

Mean Squared Error: 176.3735387676195
Mean Absolute Error: 9.654193995146553
Root Mean Squared Error: 13.280569971489156
R2 Score: 0.9940853862137997


Now that we have performed a Linear regression together with training and test data and evaluated it. It is time to perform a linear regression witht the XgBoost package.  

In [20]:
from xgboost import XGBRegressor
xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)